In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken '' # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

In [ ]:
# Напишите функцию, с Jinja выражением, которая выводит четную последовательность от 1 до N-1.
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
from jinja2 import Template

def jinja(x):
  x = int(x)
  t = Template("{% for i in range(1, x) if i%2 == 0 %}{{i}} {% endfor %}")
  print(t.render(x=x))

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))

def print_context(**context):
    context['ti'].xcom_push(key='context_len', value=str(len(context)))

run_this = PythonOperator(
    task_id='print_the_context',
    python_callable=print_context,
    dag=dag,
)

ji = PythonOperator(
    task_id='print_ji',
    python_callable=jinja,
    op_kwargs={'x': 10},
    dag=dag,
)